In [74]:
import numpy as np
from src.Model import Model
from src.Convolution import Convolution
from src.Dense import Dense

In [70]:
model = Model()
model.add(Dense(num_units=5, activation_function="relu"))
model.add(Dense(num_units=3, activation_function="relu"))
model.add(Dense(num_units=1, activation_function="sigmoid"))

In [71]:
X = np.array([
    [1, 2, 3, 4, 5],
    [10, 20, 30, 40, 25],
    [3, 6, 7, 2, 4],
    [20, 10, 40, 25, 30],
    [4, 5, 6, 7, 8]
])
y = np.array([[0], [1], [0], [1], [0]])

In [ ]:
model.fit(X, y, epochs=10, batch_size=3, learning_rate=0.1)

In [73]:
model.layers[0].weights

array([[ 0.51067987,  0.92659747,  0.29598352, -1.05021232, -0.35579706],
       [ 1.03118538, -1.76681611,  1.17066132,  0.24575175,  1.04087784],
       [-0.35887376,  0.71598444,  1.72680438,  0.17649652, -1.45018668],
       [-2.25687601,  0.29754216,  0.82527549, -0.27954341,  0.27580639],
       [-1.67569705, -0.08423213, -0.18591362, -0.71175054,  0.38311937]])

In [21]:
model.predict([[4, 5, 6, 7, 8]])

array([[0.00631896]])

## Convolution

In [1]:
import numpy as np
from src.Model import Model
from src.Convolution import Convolution
from src.Dense import Dense

In [2]:
Xm = np.array([
    [[[1,2,3],[4,5,6],[7,8,9]], [[10,11,12],[13,14,15],[16,17,18]], [[19,20,21], [22,23,24], [25,26,27]]]
])
ym = np.array([
    [[[1],[1]],
     [[1],[1]]]
])
input_shape = Xm[0].shape
m2 = Model()
m2.add(Convolution(input_size=input_shape, filter_size=(2, 2), num_filters=1, padding_size=0, stride=1))

In [3]:
m2.fit(Xm, ym, epochs=10)

ValueError: setting an array element with a sequence.